### Random Forest on Fn Platform


#### by Kamal Pandala

##### on June 19th 2018

### Random Forest

1. The commercial implementation of the Random Forest (RF) algorithm is not capable of online learning.

2. The scikit-learn's implementation must have the `partial_fit` method implemented to support this.

3. There are research papers discussing an online approach to implementing RF algorithm. However, this seems out of scope given our timeline.

4. As online learning is not supported, the algorithm requires the entire dataset to be available at any given moment to conduct training.

5. This poses a serious requirement on the available memory for large datasets. *This requires the flexibility of configuration of memory limits of a single function.*

6. Also, this removes the possibility of implementing any kind of concurrency based on splitting the data.

### Scikit-learn's implementation vs. implementation from scratch

1. The fundamental unit of computation of the RF algorithm is a Decision Tree (DT).

2. The goal of the algorithm is to build, say `n` DTs on different subsamples (with or without replacement) of the input dataset.

3. These `n` DTs are used for prediction and the results are aggregated based on maximum vote.

4. The algorithm was implemented from scratch based on [link](https://machinelearningmastery.com/implement-random-forest-scratch-python/ "How to Implement Random Forest From Scratch in Python").

5. This implementation is considerably slower compared to the scikit-learn's implementation on constructing a single DT. The time taken by scikit-learn's RF was 1.5 mins compared to the scratch implementation which could not complete within 30 mins on training a dataset of size 1.8 GB in-memory.

6. Also, the scikit-learn's implementation has various validation check and optimisations with certain modules written in C under the hood.

7. To achieve the same preformance, it would require more time and effort with possible code duplication from the scikit-learn's implementation.

### Parallelising scikit-learn's RF algorithm's training phase

1. Scikit-learn's implementation runs the building of say, `n` trees in parallel by making use of the available cores and is restricted to a single machine.

2. The data is replicated in memory `n` times for a copy to be available for proccessing by each core.

3. For example, to train a 2 GB in-memory dataset using 800 DTs for a RF model on a machine with 8 cores and 64 GB RAM, the maximum concurrency achieved can only be 8 DTs at unit time and would take 100 such rounds to complete.

4. Parallelising scikit-learn's RF training using Fn platform is straightforward and is a high-level approach.

5. To meet the requirement of building `n` DTs, we invoke `m` Fn functions with each function building `k` trees such that, `n = m * k`.

6. *However, the values of `m` and `k` are to be smartly inferred by the orchestrator `train-flow` than being hardcoded.*

### Parallelising scikit-learn's RF algorithm's training phase (contd.)

7. This can be achieved by reading the Fn function limits imposed either on the app or the account or provided by the user. An example would be, "this app or account can provision a maximum of 100 functions". Then, we can have 100 functions invoked to achieve a concurrency of 100 DTs at unit time and it would take 8 such rounds to complete. (`m = 100` and `k = 8`)

8. The output of each training process would be a model file persisted onto disk. We would end up with `m` such model files. Scikit-learn combines these `m` model files into one file. However, we are keeping it separate to help with parallelising the prediction phase.

9. This phase would have a `train-flow` Fn workflow with a `train` Fn function.

### Parallelising scikit-learn's RF algorithm's prediction phase

1. There are 2 possibilites of parallelising in the prediction phase: based on either models or rows of the data. However, given that the number of rows in a dataset could be very large, it is pragmatic to parallelise based on available models that would rarely exceed the order of thousands.

2. For each of the model file out of `m` files generated in the training phase, we invoke a Fn function that runs prediction for the new data using the chosen model and outputs the predictions. We end up with `m` such predictions with their corresponding votes/probabilities.

3. These `m` predictions are to be aggregated by averaging out the probabilities or counting the maximum votes.

4. This phase would have a `predict-workflow` Fn workflow with `predict` and `aggregate` Fn functions.

### Architecture

!["Random Forest on Fn Platform Parallel Architecture"](img/rf-parallel-architecture.jpg)

### Interface

1. The interface can be written as a Python package that can be imported and used from anywhere. It would serve as a client to communicate with the deployed `rf-parallel` app on the Fn server.

2. The user has to specify the endpoint and some form of authentication/authorization token. This validation can be under the root path.

3. The user can submit details such as location of dataset, model and output; parameters to the classifier and other configuration options which will be posted as JSON from the client.

4. The wrapper will take care of posting the JSON data to the required route and displays the repsonse to the user.

5. It will closely emulate the structure and naming conventions of scikit-learn for easier adoption.

### Queries

1. Dataset is downloaded from remote (over intranet) or cloud storage (over internet) onto the function's local storage and loaded into memory. Compared to the traditional use-case where the dataset is downloaded once and replicated in memory, this approach downloads the dataset once for every function which leads to the expenditure of a lot of network bandwidth. Any further way to optimise this ?

2. Is it possible to programatically build and deploy Fn apps by communicating with the Fn server ? (In this case, the data could be downloaded once and built with the image.)

2. Is Fn Flow capable of waiting for async functions ? If so, how is Fn Flow charged for the user ? As we know that the functions are usually charged for their execution time. Is there any timeout for Fn Flow ?

3. Is it possible to configure or alter memory limits beyond existing maximum constraints ?

4. Is it possible to manipulate JSON data within Java Flow entry function without the use of defining a POJO template ?

### Points to be addressed

1. If dataset is being downloaded from cloud storage over internet, it should be cached at remote storage and then used to serve the functions to reduce internet bandwidth consumption.

2. The loading of the data within a function is achieved using the `pandas` library. Should check if switching to C engine under the hood for `pandas` library speeds up the loading time without loss of any necessary features ?

3. To avoid out-of-memory errors, should check if using `numpy`'s `memmap` to utilise data directly from on disk without loading into memory is efficient ? How much does it impact performance ?

### Pending Work

1. `aggregate` function under `predict-flow`. (0.5 days)
2. `train-flow` and `predict-flow` workflows. (1 day)
3. Tuning phase, either `GridSearchCV` or `RandomisedSearchCV` or both based on available time. (2 days)
4. Validation checks, code refactoring and standardising naming conventions. (1 day)

### `train-flow`

1. In the handler, fetch the input JSON data as `data`.
2. Set `n = data.params.n_estimators`, `m = function_limit` and `k = n / m`.
3. Modify JSON with `data.params.n_estimators = k`, `data.params.n_jobs = 1` and `uuid` values for the location prefix.
4. Invoke `m` `train` functions concurrently with `fn_num` value set in the passed JSON object `data`.
5. Once all `m` functions have finished executing, return location of model files to the user.

### `predict-flow`

1. In the handler, fetch the input JSON as `data`.
2. Fetch location of model files from `data`, estimate number of files in the directory and set it as `m`.
3. Invoke `m` `predict` functions concurrently with `fn_num` value set in the passed JSON object `data`.
4. Once all `m` functions have finished executing, invoke `aggregate` function with location of intermediate prediction files.
5. Once `aggregate` function has finished executing, return location of predicted file to the user.

## Thank You!